In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/embeddings"))

# Any results you write to the current directory are saved as output.

In [ ]:
from torchviz import make_dot

In [ ]:
import torch
from torchtext import data

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.utils.data_utils import GeneratorEnqueuer 

import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.utils.data_utils import GeneratorEnqueuer  # We only want this for multithreaded 

from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence
from torch import Tensor

Load data

In [ ]:
main_df = pd.read_csv("../input/train.csv")
print(main_df.shape)

keep = [len(x[:-1].split()) > 0 for x in main_df["question_text"]]
main_df = main_df[keep]

main_df = main_df.sample(n=main_df.shape[0])
main_df = main_df[["question_text", "target"]]
main_df[main_df['target']==1].head()

In [ ]:
pd.options.display.max_colwidth = 3000
print(main_df.loc[:,('question_text','target')].head(10))

In [ ]:
def clean_text_add_features (dataframe):
    text=dataframe.copy()   
    
    text['specialchar']=text['question_text'].apply(lambda x: len([l for l in str(x) if not (str(l).isalnum() or str(l).isspace())])/len([l for l in str(x)]))
    text['questioncount']=text['question_text'].apply(lambda x: len([l for l in str(x) if l == '?']))
    
    #remove url
    text['question_text']=  text['question_text'].apply(lambda x: re.sub('[a-zA-Z]+[.]{1}[a-zA-Z0-9]+[.]*[a-zA-Z0-9//]*',' ',str(x)))
    text['question_text']=  text['question_text'].apply(lambda x: re.sub('[^a-zA-Z ]','',str(x)))
    
    #feature extraction
    text['words']= text['question_text'].apply(lambda x: str(x[0:]).split())
    text['question_text']= text['words'].apply(lambda x: ' '.join(x))
    text['wordcount']=text['words'].apply(lambda x: len(x))
    
    #filter to include only those rows where the wordcount is >0
    text=text[(text.wordcount>0)]
    
    
    text['avewordlength']=text['words'].apply(lambda x: sum([len(l) for l in x])/len([len(l) for l in x]))
    text['firstpersonwordcount']=text['words'].apply(lambda x: len([l for l in x if (str(l).lower()=='i' or str(l).lower()=='we' or str(l).lower()=='my')])/len([w for w in x]))
    text['uniquewords']=text['words'].apply(lambda x: len(set(x))/len(x))
    text['capitalizedwords']= text['words'].apply(lambda x: len([w for w in x if not w.islower()])/len([w for w in x]))
    #text['question_text']=text['question_text'].apply(lambda x: str(x).lower())
    return text

In [ ]:
main_df = clean_text_add_features(main_df)

import string
main_df['question_text'] = main_df['question_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
#Max question length
main_df.columns
#print(max(main_df['question_text'].apply(lambda x: len(x.split(' ')))))



In [ ]:
main_df_1 = main_df[main_df['target'] == 1]
main_df_0 = main_df[main_df['target'] == 0]

from sklearn.model_selection import train_test_split
train_1, test_1 = train_test_split(main_df_1, test_size = 0.1)
train_0, test_0 = train_test_split(main_df_0, test_size = 0.1)

train_df = train_1.append(train_0)
val_df = test_1.append(test_0)

In [ ]:
from tqdm import tqdm_notebook as tqdm
import math

embeddings_index = {}
f = open('../input/embeddings/glove.840B.300d/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
def text_to_array(text):
    empyt_emb = np.zeros(300)
    question = text.split()[:120]
    embeds = [embeddings_index.get(x, empyt_emb) for x in question] #get embedding else 0 if word DNE in Glove
    seq_len = len(embeds) #len of text
    embeds+= [empyt_emb] * (120 - len(embeds))
    return np.array(embeds), seq_len


embeddings = [text_to_array(X_text) for X_text in tqdm(val_df["question_text"][:5000])]
val_x, val_xlen = zip(*embeddings) #train_x is the a list of 120 vectors each of len 300

val_x2 = val_df.loc[:,(val_df.columns!='question_text') | (val_df.columns!='target') ]

In [ ]:
# For dynamic LSTM the values have to be ordered.
sorder = np.argsort(val_xlen)[::-1]

val_xlen = np.array(val_xlen)[sorder]
val_x = np.array(val_x)[sorder]
val_y = np.array(val_df["target"][:5000])[sorder]
val_x2 = val_df[['specialchar', 'questioncount',
       'wordcount', 'avewordlength', 'firstpersonwordcount', 'uniquewords',
       'capitalizedwords']]
val_x2 = np.array(val_x2)[sorder]

In [ ]:
# Train data providers
batch_size = 128
def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    counter = 0
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        counter+=1
        print('Epoch no: ', counter)
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 0]
            text_arr, text_len = zip(*[text_to_array(text) for text in texts])
            sorder = np.argsort(text_len)[::-1]
            text_arr = np.array(text_arr)[sorder]
            text_len = np.array(text_len)[sorder]
            text_meta = train_df[['specialchar', 'questioncount',
       'wordcount', 'avewordlength', 'firstpersonwordcount', 'uniquewords',
       'capitalizedwords']]
            text_meta = np.array(text_meta)[sorder]
            yield text_arr, text_len, text_meta, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])[sorder]

## Training the model

In [ ]:
class BiLSTM_model(nn.Module):
    def __init__(self, embedding_size=300, hidden_size=128):
        super(BiLSTM_model, self).__init__()
        self.lstm = nn.LSTM(300, hidden_size, 
                            batch_first=True, num_layers=2,
                            bidirectional=True)
        
        #self.fc = nn.Linear(hidden_size*2, 1)
        
        self.fc2 = nn.Linear(hidden_size*2, 64)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(64, 13)
        self.fc4 = nn.Linear(13 + 7, 1)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, xmeta, input_lengths):
        x = pack_padded_sequence(x, input_lengths, batch_first=True)
        out = self.lstm(x)
        out = out[1][1][2:] # TODO check if :2 really is FW and BW of last layer!
        out = out.transpose(0,1)
        out = out.flatten(start_dim=1)
        out = self.relu(self.fc2(out))
        
        out = self.relu(self.fc3(out))
        
        newvar = torch.cat((out, xmeta), 1)
        final_out = self.fc4(newvar)
        return self.sigmoid(final_out).flatten()
        # return memory

In [ ]:
# class BiLSTM_model(nn.Module):
#     def __init__(self, embedding_size=300, hidden_size=128):
#         super(BiLSTM_model, self).__init__()
#         self.lstm = nn.LSTM(300, hidden_size, 
#                             batch_first=True, num_layers=3,
#                             bidirectional=True)
        
#         self.fc = nn.Linear(hidden_size*4, 1)
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x, input_lengths):
#         x = pack_padded_sequence(x, input_lengths, batch_first=True)
#         out = self.lstm(x)
#         out = out[1][1][2:] # TODO check if :2 really is FW and BW of last layer!
#         out = out.transpose(0,1)
#         out = out.flatten(start_dim=1)
#         out = self.fc(out)
#         return self.sigmoid(out).flatten()
#         # return memory

In [ ]:
model = BiLSTM_model()
#model.cuda()
model.train()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
 #Binary cross entropy loss

my_generator = GeneratorEnqueuer(batch_gen(train_df))
my_generator.start()
mg =  my_generator.get()

In [ ]:
#!pip install git+https://github.com/szagoruyko/pytorchviz

#model_parameters = filter(lambda p: p.req, model.parameters())
#params = sum([np.prod(p.size()) for p in model_parameters])

sum([np.prod(p.size()) for p in model_parameters])
# for i, (x, xlen,xmeta, y) in tqdm(enumerate(mg)):
#     break
# x = Tensor(x)
# xmeta = Tensor(xmeta)
# from torchviz import make_dot, make_dot_from_trace
# make_dot(model(x, xmeta, xlen),model.named_parameters())

In [ ]:
with torch.onnx.set_training(model, False):
    trace, _ = torch.jit.get_trace_graph(model, args=([x, xmeta, xlen,]))
make_dot_from_trace(trace)

In [ ]:
def val_gen(batch_size=256):
    n_batches = math.ceil(len(val_x) / batch_size)
    for idx in range(n_batches):
        xb = val_x[idx *batch_size:(idx+1) * batch_size]
        xlb = val_xlen[idx *batch_size:(idx+1) * batch_size]
        xmeta = val_x2[idx *batch_size:(idx+1) * batch_size]
        yb = val_y[idx *batch_size:(idx+1) * batch_size]
        yield xb, xlb, xmeta, yb

In [ ]:
# def val_gen(batch_size=256):
#     n_batches = math.ceil(len(val_df['question_text'][:20000]) / batch_size)

#     for idx in range(n_batches):
#         embeddings = [text_to_array(X_text) for X_text in tqdm(val_df["question_text"][idx *batch_size:(idx+1) * batch_size])]
#         val_x, val_xlen = zip(*embeddings) #train_x is the a list of 120 vectors each of len 300
        
#         sorder = np.argsort(val_xlen)[::-1]

#         val_xlen = np.array(val_xlen)[sorder]
#         val_x = np.array(val_x)[sorder]
#         val_y = np.array(val_df["target"][idx *batch_size:(idx+1) * batch_size])[sorder]
#         val_x2 = val_df[['specialchar', 'questioncount', 'wordcount', 'avewordlength', 'firstpersonwordcount', 'uniquewords',
#        'capitalizedwords']]
#         val_x2 = np.array(val_x2)[sorder]
#         xb = val_x
#         xlb = val_xlen
#         xmeta = val_x2
#         yb = val_y
#         yield xb, xlb, xmeta, yb

In [ ]:
losses = []
device = torch.device('cuda')
val_losses = []
for i, (x, xlen,xmeta, y) in tqdm(enumerate(mg)):
    optimizer.zero_grad()
    

    x = Tensor(x).cuda()
    xmeta = Tensor(xmeta).cuda()
    y_pred = model(x, xmeta, xlen)
    #bcloss = nn.BCELoss(weight = Tensor((y*1)+1).cuda())
    bcloss = nn.BCELoss()
    loss = bcloss(y_pred, Tensor(y).cuda())
    loss.backward()
    
    losses.append(loss.data.cpu().numpy())
    optimizer.step()
    
    if (i + 1) % 1000 == 0:
        print("Iter: {}".format(i+1))
        print("\tAverage training loss: {:.5f}".format(np.mean(losses)))
        # Evaluate F1 on validation set
        model.eval()
        all_preds = []
        for x, xlen, xmeta, y in val_gen():
            y_pred = model(Variable(Tensor(x)).cuda(),Variable(Tensor(xmeta)).cuda(), xlen)
            all_preds.extend(y_pred.cpu().data.numpy())
            bcloss = nn.BCELoss()
            val_losses.append(bcloss(y_pred, Tensor(y).cuda()))
        score = f1_score(val_y, np.array(all_preds).flatten() > 0.5)
        print("\tVal F1 score: {:.5f}".format(score))
        model.train()
    if  i == 10000:
        print("Reducing LR")
        for g in optimizer.param_groups:
            g['lr'] = 0.0001
    if (i + 1) % 12000 == 0:  # We are done
        break

In [ ]:
# for index,i in enumerate(val_losses):
#     val_losses[index] = val_losses[index].data.cpu().numpy()
    
# val_losses_list = np.array(val_losses).tolist()
# losses_list = np.array(losses).tolist()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.plot(val_losses_list,color = 'red')
# plt.plot(losses_list, color = 'blue')

# plt.show()

In [ ]:
# len(val_losses_list)

In [ ]:

batch_size = 256

def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        ids = test_df["qid"][i*batch_size:(i+1)*batch_size]
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr, text_len = zip(*[text_to_array(text) for text in texts])
        sorder = np.argsort(text_len)[::-1]
        
        text_arr = np.array(text_arr)[sorder]
        text_len = np.array(text_len)[sorder]
        text_meta = test_df[['specialchar', 'questioncount',
       'wordcount', 'avewordlength', 'firstpersonwordcount', 'uniquewords',
       'capitalizedwords']]
        text_meta = np.array(text_meta)[sorder]
        ids = np.array(ids)[sorder]
        yield text_arr, text_meta, text_len, ids
    
test_df = pd.read_csv("../input/test.csv")

test_df = clean_text_add_features(test_df)
test_df['question_text'] = test_df['question_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

all_preds = []
all_ids = []

model.eval()
for x, xmeta, xlen, xid in tqdm(batch_gen(test_df)):
    preds = model(Variable(Tensor(x).cuda()),Variable(Tensor(xmeta).cuda()), xlen)
    preds = preds.data.cpu().numpy()
    all_preds.extend(preds)
    all_ids.extend(xid)

In [ ]:
y_te = (np.array(all_preds).flatten() > 0.5).astype(np.int)

submit_df = pd.DataFrame({"qid": all_ids, "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)